In [2]:
csize(a)=(ndims(a)==1 ? size(a) : size(a)[1:end-1])
csize(a,n)=tuple(csize(a)..., n) # size if you had n columns
clength(a)=(ndims(a)==1 ? length(a) : stride(a,ndims(a)))
ccount(a)=(ndims(a)==1 ? 1 : size(a,ndims(a)))
csub(a,i)=(ndims(a)==1 ? error() : sub(a, ntuple(i->(:), ndims(a)-1)..., i))
cget(a,i)=(ndims(a)==1 ? error() : getindex(a, ntuple(i->(:), ndims(a)-1)..., i))
size2(y)=(nd=ndims(y); (nd==1 ? (length(y),1) : (stride(y, nd), size(y, nd)))) # size as a matrix
size2(y,i)=size2(y)[i]

size2 (generic function with 2 methods)

In [3]:
function minibatch(x, y, batchsize)
    data = Any[]
    for i=1:batchsize:ccount(x)
        j=min(i+batchsize-1,ccount(x))
        push!(data, (cget(x,i:j), cget(y,i:j)))
    end
    return data
end

minibatch (generic function with 1 method)

In [4]:
function train(w, dtrn, dtst; lr=.5, epochs=10)
    for epoch=1:epochs
        @printf("\ntrn-acc=%6.4f,tst-acc=%6.4f\n",
        accuracy(w,dtrn),accuracy(w,dtst))
        for (x,y) in dtrn
            g = lossgradient(w, x, y)
            for i in 1:length(w)
                w[i] -= lr*g[i]
            end
        end   
    end
end

train (generic function with 1 method)

In [5]:
function err(w, dtst)
    nloss = ninstance = 0
    for (x, y) in dtst
        nloss += loss(w,x,y)
        ninstance += 1
    end
    return sqrt(nloss/ninstance)
end

err (generic function with 1 method)

In [6]:
function dataprovider(A)    
    map!(view(A,:,5)) do x
        x = x==1?[1,0,0]:(x==0?[0,1,0]:[0,0,1])
    end
    
    x0 = Float32[diff(A[1,3][end-2Tb:end])/100;diff(A[1,4][end-Tb:end]);
        diff(A[1,1][end-6Tb:end]);diff(A[1,2][end-6Tb:end])]
    y0 = Float32.(A[1,5])

    x = zeros(length(x0),size(A,1))
    y = zeros(length(y0),size(A,1))
    for (t,i) in enumerate(randcycle(size(A,1)))
        x[:,t] = Float32[diff(A[i,3][end-2Tb:end])/100;diff(A[i,4][end-Tb:end]);
            diff(A[i,1][end-6Tb:end]);diff(A[i,2][end-6Tb:end])]
        y[:,t] = Float32.(A[i,5])
    end

    data = minibatch(atype(x),atype(y),100)
    dtrn = data[1:4*end÷5]
    dtst = data[4*end÷5+1:end]
    return dtrn,dtst
end

dataprovider (generic function with 1 method)

In [ ]:
function dataprovider2(A)
    A = copy(A)
    A[:,end] += 1


    x0 = Float32[diff(A[1,3][end-2Tb:end])/100;diff(A[1,4][end-Tb:end]);
        diff(A[1,1][end-6Tb:end]);diff(A[1,2][end-6Tb:end])]
    y0 = Float32.(A[1,5])

    x = zeros(length(x0),size(A,1))
    y = zeros(length(y0),size(A,1))
    for (t,i) in enumerate(randcycle(size(A,1)))
        x[:,t] = Float32[diff(A[i,3][end-2Tb:end])/100;diff(A[i,4][end-Tb:end]);
            diff(A[i,1][end-6Tb:end]);diff(A[i,2][end-6Tb:end])]
        y[:,t] = Float32.(A[i,5])
    end

    xtrn = x[:,1:4*end÷5]; ytrn = y[:,1:4*end÷5]
    xtst = x[:,4*end÷5+1:end]; ytst = y[:,4*end÷5+1:end]
    return xtrn',vec(ytrn),xtst',vec(ytst)
end

In [7]:
function profit(signal,buy_price,sell_price)
    count = 0
    state = 0
    price = 0
    profits = 0
    for t=1:length(signal)
        if signal[t] == 1   
            if state == 0
                adv_price = sell_price[t]   
                state = 1
                price = adv_price              
            elseif state == -1
                adv_price = sell_price[t]
                profits += price - adv_price - 0.4
                count += 1
                state = 0
                price = 0              
            end
        elseif signal[t] == -1
            if state == 0
                adv_price = buy_price[t]   
                state = -1
                price = adv_price
            elseif state == 1
                adv_price = buy_price[t]
                profits += -price + adv_price - 0.4
                count += 1
                state = 0
                price = 0
            end
        end
    end
    return count,profits
end 

profit (generic function with 1 method)

In [8]:
function weights(data,h...; atype=Array{Float32}, winit=0.1)
    x0,y0 = data[1]
    w = Any[]
    x = size(x0,1)
    for y in [h..., size(y0,1)]
        push!(w, convert(atype, winit*randn(y,x)))
        push!(w, convert(atype, zeros(y, 1)))
        x = y
    end
    return w  
end

weights (generic function with 1 method)

In [ ]:
function entropy(p)
    p0 = ones(p)/length(p)
    sum(-p.*log(p+1e-20))/sum(-p0.*log(p0+1e-20))
end